In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [2]:
import torch
import torchvision
from torchvision import datasets
from torchvision import transforms as T # for simplifying the transforms
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader, sampler, random_split
from torchvision import models

In [3]:
import timm
from timm.loss import LabelSmoothingCrossEntropy #

In [4]:
# remove warnings
import warnings
warnings.filterwarnings("ignore")

In [5]:
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
import sys
from tqdm import tqdm
import time
import copy

In [7]:
def get_classes(data_dir):
    all_data = datasets.ImageFolder(data_dir)
    return all_data.classes

In [8]:
def get_data_loaders(data_dir, batch_size, train = False):
    if train:
        #train
        transform = T.Compose([
            T.RandomHorizontalFlip(),
            T.RandomVerticalFlip(),
            T.Resize(256),
            T.CenterCrop(224),
            T.ToTensor(),
            T.Normalize(timm.data.IMAGENET_DEFAULT_MEAN, timm.data.IMAGENET_DEFAULT_STD), # imagenet means
           ])
        train_data = datasets.ImageFolder(os.path.join(data_dir, "train/"), transform = transform)
        train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=4)
        return train_loader, len(train_data)
    else:
        # val/test
        transform = T.Compose([ # We dont need augmentation for test transforms
            T.Resize(256),
            T.CenterCrop(224),
            T.ToTensor(),
            T.Normalize(timm.data.IMAGENET_DEFAULT_MEAN, timm.data.IMAGENET_DEFAULT_STD), # imagenet means
        ])
        val_data = datasets.ImageFolder(os.path.join(data_dir, "val/"), transform=transform)
        test_data = datasets.ImageFolder(os.path.join(data_dir, "val/"), transform=transform)
        val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True, num_workers=4)
        test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True, num_workers=4)
        return val_loader, test_loader, len(val_data), len(test_data)

In [9]:
#dataset_path = 'C:/Users/CON3ADOLEA/Documents/deit-main/data'
dataset_path = 'M:/VieApp/biophilic_training_dataset'

In [10]:
(train_loader, train_data_len) = get_data_loaders(dataset_path, 128, train=True)
(val_loader, test_loader, valid_data_len, test_data_len) = get_data_loaders(dataset_path, 32, train=False)

In [11]:
classes = get_classes("M:/VieApp/biophilic_training_dataset/train/")
print(classes, len(classes))

['awe', 'biomorphic_shapes', 'complexity_in_order', 'connection_with_nature', 'mystery', 'natural_materials', 'natural_organisation', 'presence_of_animals', 'presence_of_plants_or_fungi', 'presence_of_water', 'refuge', 'risk', 'unimpeded_views', 'varying_light'] 14


In [12]:
dataloaders = {
    "train": train_loader,
    "val": val_loader
}
dataset_sizes = {
    "train": train_data_len,
    "val": valid_data_len
}

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [14]:
HUB_URL = "SharanSMenon/swin-transformer-hub:main"
MODEL_NAME = "swin_tiny_patch4_window7_224"
# check hubconf for more models.
model = torch.hub.load(HUB_URL, MODEL_NAME, pretrained=True) # load from torch hub

Using cache found in C:\Users\CON3ADOLEA/.cache\torch\hub\SharanSMenon_swin-transformer-hub_main


In [15]:
for param in model.parameters(): #freeze model
    param.requires_grad = False

n_inputs = model.head.in_features
model.head = nn.Sequential(
    nn.Linear(n_inputs, 512),
    nn.ReLU(),
    nn.Dropout(0.3),
    nn.Linear(512, len(classes))
)
model = model.to(device)
#print(model.head)

In [16]:
criterion = LabelSmoothingCrossEntropy()
criterion = criterion.to(device)
optimizer = optim.AdamW(model.head.parameters(), lr=0.001)

In [17]:
# lr scheduler
exp_lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.97)

In [18]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=100):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print("-"*10)
        
        for phase in ['train', 'val']: # We do training and validation phase per epoch
            if phase == 'train':
                model.train() # model to training mode
            else:
                model.eval() # model to evaluate
            
            running_loss = 0.0
            running_corrects = 0.0
            
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)
                
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'): # no autograd makes validation go faster
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1) # used for accuracy
                    loss = criterion(outputs, labels)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
            if phase == 'train':
                scheduler.step() # step at end of epoch
            
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc =  running_corrects.double() / dataset_sizes[phase]
            
            print("{} Loss: {:.4f} Acc: {:.4f}".format(phase, epoch_loss, epoch_acc))
            
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict()) # keep the best validation accuracy model
        print()
    time_elapsed = time.time() - since # slight error
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print("Best Val Acc: {:.4f}".format(best_acc))
    
    model.load_state_dict(best_model_wts)
    return model

In [54]:
model_ft = train_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs=300)

Epoch 0/299
----------


100%|██████████| 82/82 [02:33<00:00,  1.87s/it]


train Loss: 2.2379 Acc: 0.2361


100%|██████████| 132/132 [06:01<00:00,  2.74s/it]


val Loss: 2.4966 Acc: 0.1724

Epoch 1/299
----------


...

val Loss: 2.7798 Acc: 0.1648

Epoch 299/299
----------


100%|██████████| 82/82 [02:33<00:00,  1.87s/it]


train Loss: 1.8322 Acc: 0.3522


100%|██████████| 132/132 [05:18<00:00,  2.42s/it]

val Loss: 2.7807 Acc: 0.1643

Training complete in 2445m 19s
Best Val Acc: 0.1798


In [19]:
model_ft = train_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs=1000)

Epoch 0/999
----------


100%|██████████| 82/82 [03:56<00:00,  2.88s/it] 


train Loss: 2.4855 Acc: 0.1592


100%|██████████| 132/132 [06:12<00:00,  2.83s/it]


val Loss: 2.5017 Acc: 0.1695

Epoch 1/999
----------


...

val Loss: 2.9036 Acc: 0.1667

Epoch 999/999
----------


100%|██████████| 82/82 [02:36<00:00,  1.91s/it]


train Loss: 1.7838 Acc: 0.3534


100%|██████████| 132/132 [06:54<00:00,  3.14s/it]

val Loss: 2.9036 Acc: 0.1667

Training complete in 9643m 46s
Best Val Acc: 0.1824


In [29]:
#model_ft = train_model(model, criterion, optimizer, exp_lr_scheduler, num_epochs=100)

Epoch 0/99
----------


100%|██████████| 27/27 [01:35<00:00,  3.55s/it]


train Loss: 1.4529 Acc: 0.5732


100%|██████████| 106/106 [01:31<00:00,  1.15it/s]


val Loss: 1.2991 Acc: 0.6385

Epoch 1/99
----------


...

val Loss: 1.0545 Acc: 0.6880

Epoch 99/99
----------


100%|██████████| 27/27 [01:37<00:00,  3.62s/it]


train Loss: 1.1424 Acc: 0.6759


100%|██████████| 106/106 [01:28<00:00,  1.20it/s]

val Loss: 1.0546 Acc: 0.6877

Training complete in 311m 53s
Best Val Acc: 0.6892


In [33]:
#model

In [32]:
model.head

Sequential(
  (0): Linear(in_features=768, out_features=512, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.3, inplace=False)
  (3): Linear(in_features=512, out_features=14, bias=True)
)

In [34]:
test_loss = 0.0
class_correct = list(0 for i in range(len(classes)))
class_total = list(0 for i in range(len(classes)))
model.eval()

for data, target in tqdm(test_loader):
    data, target = data.to(device), target.to(device)
    with torch.no_grad(): # turn off autograd for faster testing
        output = model(data)
        loss = criterion(output, target)
    test_loss = loss.item() * data.size(0)
    _, pred = torch.max(output, 1)
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.cpu().numpy())
    if len(target) == 32:
        for i in range(32):
            label = target.data[i]
            class_correct[label] += correct[i].item()
            class_total[label] += 1

test_loss = test_loss / test_data_len
print('Test Loss: {:.4f}'.format(test_loss))
for i in range(len(classes)):
    if class_total[i] > 0:
        print("Test Accuracy of %5s: %2d%% (%2d/%2d)" % (
            classes[i], 100*class_correct[i]/class_total[i], np.sum(class_correct[i]), np.sum(class_total[i])
        ))
    else:
        print("Test accuracy of %5s: NA" % (classes[i]))
print("Test Accuracy of %2d%% (%2d/%2d)" % (
            100*np.sum(class_correct)/np.sum(class_total), np.sum(class_correct), np.sum(class_total)
        ))

100%|██████████| 106/106 [01:31<00:00,  1.16it/s]

Test Loss: 0.0023
Test Accuracy of Varying_light: 73% (114/156)
Test Accuracy of   awe: 76% (270/353)
Test Accuracy of biomorphic shapes: 70% (186/263)
Test Accuracy of complexity_in_order: 91% (182/199)
Test Accuracy of connection_with_nature: 80% (161/200)
Test Accuracy of mystery: 76% (153/199)
Test Accuracy of natural materials: 53% (178/335)
Test Accuracy of natural organisation: 60% (231/384)
Test Accuracy of presence of animals: 84% (175/207)
Test Accuracy of presence_of_plants_or_fungi: 41% (63/153)
Test Accuracy of presence_of_water: 67% (129/190)
Test Accuracy of refuge: 51% (94/181)
Test Accuracy of  risk: 76% (163/214)
Test Accuracy of unimpeded views: 66% (216/326)
Test Accuracy of 68% (2315/3360)


In [35]:
# our model earns 93% test accuracy, which is very high. lets save it
example = torch.rand(1, 3, 224, 224)
traced_script_module = torch.jit.trace(model.cpu(), example)
traced_script_module.save("biophilic_swin_2.pt")